In [47]:
# 라이브러리 import
import glob
import cv2
import pandas as pd
import matplotlib.pyplot as plt
import argparse
import random
import os
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from tqdm import tqdm
from copy import deepcopy

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

import torchvision.models as models

from sklearn import preprocessing
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

In [4]:
import timm
from pprint import pprint
model = timm.create_model('efficientnet_b4',pretrained=True)
model

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/efficientnet_b4_ra2_320-7eb33cd5.pth" to /opt/ml/.cache/torch/hub/checkpoints/efficientnet_b4_ra2_320-7eb33cd5.pth


EfficientNet(
  (conv_stem): Conv2d(3, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn1): BatchNormAct2d(
    48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
    (drop): Identity()
    (act): SiLU(inplace=True)
  )
  (blocks): Sequential(
    (0): Sequential(
      (0): DepthwiseSeparableConv(
        (conv_dw): Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=48, bias=False)
        (bn1): BatchNormAct2d(
          48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): SiLU(inplace=True)
        )
        (se): SqueezeExcite(
          (conv_reduce): Conv2d(48, 12, kernel_size=(1, 1), stride=(1, 1))
          (act1): SiLU(inplace=True)
          (conv_expand): Conv2d(12, 48, kernel_size=(1, 1), stride=(1, 1))
          (gate): Sigmoid()
        )
        (conv_pw): Conv2d(48, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn2): BatchNormAct2d(
    

In [48]:
# 파라미터 정의
parser = argparse.ArgumentParser()
parser.add_argument('--epochs', type=int, default=60)
parser.add_argument('--lr', type=float, default=0.001)
parser.add_argument('--batch_size', type=int, default=32)
parser.add_argument('--seed', type=int, default=999)
parser.add_argument('--img_size', type=int, default=224)
parser.add_argument('--device', default='cuda')
parser.add_argument('--beta', default=1)

args = parser.parse_args('')
args

Namespace(batch_size=32, beta=1, device='cuda', epochs=60, img_size=224, lr=0.001, seed=999)

In [49]:
# 시드 고정
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(args.seed)

In [50]:
df = pd.read_csv('./data/train_repaired.csv')

In [51]:
# 레이블 변경
le = preprocessing.LabelEncoder()
df['artist'] = le.fit_transform(df['artist'].values)
df.head()

,id,img_path,artist
0,0,./train/0000.jpg,9
1,1,./train/0001.jpg,48
2,2,./train/0002.jpg,7
3,3,./train/0003.jpg,10
4,4,./train/0004.jpg,24


In [26]:
train_df = pd.DataFrame()
val_df = pd.DataFrame()
g = df.groupby('artist', group_keys=False)
for name, p in g:
    train, val, _, _ = train_test_split(p, p, test_size=0.2, random_state=args.seed)
    train_df = pd.concat((train_df, train))
    val_df = pd.concat((val_df, val))

In [6]:
g = df.groupby('artist', group_keys=False)
df_sample = pd.DataFrame()
for _ in range(15):
  df_sample = pd.concat([df_sample, g.apply(lambda x: x.sample(21))])

In [53]:
train_df, val_df, _, _ = train_test_split(df, df['artist'].values, test_size=0.2, random_state=args.seed, stratify=df.artist.values)

In [55]:
train_df.groupby('artist').count()

,id,img_path
artist,,
0,176,176
1,132,132
2,106,106
3,59,59
4,106,106
5,51,51
6,26,26
7,47,47
8,40,40


In [57]:
val_df.groupby('artist').count()

,id,img_path
artist,,
0,44,44
1,33,33
2,26,26
3,15,15
4,26,26
5,13,13
6,6,6
7,12,12
8,10,10


In [8]:
def get_data(df, infer=False):
    if infer:
        return df['img_path'].values
    return df['img_path'].values, df['artist'].values

In [9]:
train_img_paths, train_labels = get_data(train_df)
val_img_paths, val_labels = get_data(val_df)

In [10]:
class CustomDataset(Dataset):
    def __init__(self, img_paths, labels, transforms=None):
        self.img_paths = img_paths
        self.labels = labels
        self.transforms = transforms

    def __getitem__(self, index):
        img_path = self.img_paths[index]
        img_path = os.path.join('./data', img_path)
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # 이미지 길이 1 / 2

        if self.labels is not None: # train
          h, w, _ = image.shape
          size = torch.tensor([h, w])
          l = min(h//2, w//2)
          inner_transform = A.Compose([A.RandomCrop(width=l, height=l)])
        else: # test
          h, w, _ = image.shape
          size = torch.tensor([2*h, 2*w])
        
        if self.transforms is not None:
            if self.labels is not None: # train
              image = inner_transform(image=image)['image']
            image = self.transforms(image=image)['image']
        
        if self.labels is not None:
            label = self.labels[index]
            return image, label, size
        else:
            return image, size
    
    def __len__(self):
        return len(self.img_paths)

In [11]:
# random flip
train_transform = A.Compose([
                            A.VerticalFlip(),
                            A.HorizontalFlip(),
                            A.RandomRotate90(),
                            A.Resize(args.img_size,args.img_size),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])

test_transform = A.Compose([
                            A.Resize(args.img_size,args.img_size),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])

In [12]:
train_dataset = CustomDataset(train_img_paths, train_labels, train_transform)
train_loader = DataLoader(train_dataset, batch_size = args.batch_size, shuffle=True, num_workers=2)

val_dataset = CustomDataset(val_img_paths, val_labels, test_transform)
val_loader = DataLoader(val_dataset, batch_size=args.batch_size, shuffle=False, num_workers=2)

In [13]:
def randBbox(size, lam):
    W = size[2]
    H = size[3]
    cut_rat = np.sqrt(1. - lam)
    cut_w = np.int32(W * cut_rat)
    cut_h = np.int32(H * cut_rat)

    # uniform
    cx = np.random.randint(W)
    cy = np.random.randint(H)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)

    return bbx1, bby1, bbx2, bby2

In [14]:
class F1Loss(nn.Module):
    def __init__(self, classes=50, epsilon=1e-7):
        super().__init__()
        self.classes = classes
        self.epsilon = epsilon

    def forward(self, y_pred, y_true):
        assert y_pred.ndim == 2
        assert y_true.ndim == 1
        y_true = F.one_hot(y_true, self.classes).to(torch.float32)
        y_pred = F.softmax(y_pred, dim=1)

        tp = (y_true * y_pred).sum(dim=0).to(torch.float32)
        tn = ((1 - y_true) * (1 - y_pred)).sum(dim=0).to(torch.float32)
        fp = ((1 - y_true) * y_pred).sum(dim=0).to(torch.float32)
        fn = (y_true * (1 - y_pred)).sum(dim=0).to(torch.float32)

        precision = tp / (tp + fp + self.epsilon)
        recall = tp / (tp + fn + self.epsilon)

        f1 = 2 * (precision * recall) / (precision + recall + self.epsilon)
        f1 = f1.clamp(min=self.epsilon, max=1 - self.epsilon)
        return 1 - f1.mean()

In [15]:
class BaseModel(nn.Module):
    def __init__(self, num_classes=len(le.classes_)):
        super(BaseModel, self).__init__()
        self.backbone = models.efficientnet_b4(weights='DEFAULT')
        self.backbone.classifier = nn.Sequential()
        self.drop = nn.Dropout(0.4, inplace=True)

        self.fc = nn.Sequential(
            nn.Linear(2, 128, bias=False),
            nn.BatchNorm1d(128),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3, inplace=True),
            nn.Linear(128, 256)
        )

        self.clf = nn.Sequential(
            nn.Linear(1792, 1024, bias=False),
            nn.BatchNorm1d(1024),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.4, inplace=True),
            nn.Linear(1024, num_classes) # + len(장르, 국가)
        )

        # rgb 평균
        
    def forward(self, x, size):
        x = self.backbone(x)
        x = self.drop(x)

        lin = self.fc(size)

        x = torch.cat((x, lin), 1) # 여기 cat부분이 cnn부분 끝나고 분류하는 layer들어가기 전에 붙인 부분입니다.
        x = self.clf(x)
        return x

In [16]:
def competition_metric(true, pred):
    return f1_score(true, pred, average="macro")

def validation(model, criterion, test_loader, device):
    model.eval()
    
    model_preds = []
    true_labels = []
    
    val_loss = []
    
    with torch.no_grad():
        for img, label, size in tqdm(test_loader):
            img, label = img.float().to(device), label.to(device)

            size = size.float().to(device)
            model_pred = model(img, size)
            # model_pred = model(img)
            
            loss = criterion(model_pred, label)
            
            val_loss.append(loss.item())
            
            model_preds += model_pred.argmax(1).detach().cpu().numpy().tolist()
            true_labels += label.detach().cpu().numpy().tolist()
        
    val_f1 = competition_metric(true_labels, model_preds)
    return np.mean(val_loss), val_f1

In [17]:
def train(model, optimizer, train_loader, test_loader, scheduler, device):
    model.to(device)

    criterion = F1Loss().to(device)
    
    best_score = 0
    best_model = None
    
    for epoch in range(1, args.epochs+1):
        model.train()
        train_loss = []
        for img, label, size in tqdm(train_loader):
            img, label = img.float().to(device), label.to(device)
            
            size = size.float().to(device)

            r = np.random.rand(1)
            if r < 0.5:
                lam = np.random.beta(args.beta, args.beta)
                rand_index = torch.randperm(img.size()[0]).to(device)
                target_a = label
                target_b = label[rand_index]
                bbx1, bby1, bbx2, bby2 = randBbox(img.size(), lam)
                img[:, :, bbx1:bbx2, bby1:bby2] = img[rand_index, :, bbx1:bbx2, bby1:bby2]
                # adjust lambda to exactly match pixel ratio
                lam = 1 - ((bbx2 - bbx1) * (bby2 - bby1) / (img.size()[-1] * img.size()[-2]))
                # compute output
                outs = model(img, size)
                loss = criterion(outs, target_a) * lam + criterion(outs, target_b) * (1. - lam)

            else:
                outs = model(img, size)
                loss = criterion(outs, label)


            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss.append(loss.item())

        tr_loss = np.mean(train_loss)
            
        val_loss, val_score = validation(model, criterion, test_loader, device)
            
        print(f'Epoch [{epoch}], Train Loss : [{tr_loss:.5f}] Val Loss : [{val_loss:.5f}] Val F1 Score : [{val_score:.5f}]')
        
        if scheduler is not None:
            scheduler.step()
            
        if best_score < val_score:
            best_model = deepcopy(model.state_dict())
            best_score = val_score
        
    return best_model

In [18]:
# ckpt = torch.load('/content/drive/MyDrive/data/model/40epoch_vit_b_32.tar')
# model = BaseModel().to(args.device)
model = BaseModel()
# model.load_state_dict(ckpt['model'])
model.eval()
optimizer = torch.optim.Adam(params = model.parameters(), lr = args.lr)
# optimizer.load_state_dict(ckpt['optim'])
# scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10, eta_min=0)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)

infer_model_param = train(model, optimizer, train_loader, val_loader, scheduler, args.device)

100%|██████████| 99/99 [00:25<00:00,  3.95it/s]


Epoch [1], Train Loss : [0.88710] Val Loss : [0.77682] Val F1 Score : [0.47308]


100%|██████████| 99/99 [00:24<00:00,  4.02it/s]


Epoch [2], Train Loss : [0.81068] Val Loss : [0.75475] Val F1 Score : [0.52661]


100%|██████████| 99/99 [00:24<00:00,  3.98it/s]


...

Epoch [58], Train Loss : [0.66423] Val Loss : [0.60401] Val F1 Score : [0.84648]


100%|██████████| 99/99 [00:24<00:00,  4.02it/s]


Epoch [59], Train Loss : [0.66039] Val Loss : [0.60053] Val F1 Score : [0.85226]


100%|██████████| 99/99 [00:25<00:00,  3.95it/s]

Epoch [60], Train Loss : [0.65968] Val Loss : [0.60247] Val F1 Score : [0.84941]


In [19]:
torch.save({
    'model': infer_model_param,
    'optim': optimizer.state_dict()
}, './models/60epoch_best_efficient_b4_3.tar')

In [20]:
test_df = pd.read_csv('./data/test.csv')

In [21]:
test_img_paths = get_data(test_df, infer=True)

In [22]:
test_dataset = CustomDataset(test_img_paths, None, test_transform)
test_loader = DataLoader(test_dataset, batch_size=args.batch_size, shuffle=False, num_workers=0)

In [23]:
def inference(model_param, test_loader, device):
    model = BaseModel().to(args.device)
    model.load_state_dict(model_param)
    model.eval()
    
    model_preds = []
    
    with torch.no_grad():
        for img, size in tqdm(iter(test_loader)):
            img = img.float().to(device)
            size = size.float().to(device)
            
            model_pred = model(img, size)
            model_preds += model_pred.argmax(1).detach().cpu().numpy().tolist()
    
    print('Done.')
    return model_preds

In [24]:
preds = inference(infer_model_param, test_loader, args.device)

100%|██████████| 396/396 [01:32<00:00,  4.27it/s]

Done.


In [25]:
preds = le.inverse_transform(preds) 

In [26]:
submit = pd.read_csv('./data/sample_submission.csv')

In [27]:
submit['artist'] = preds
submit.head()

,id,artist
0,TEST_00000,Jan van Eyck
1,TEST_00001,Amedeo Modigliani
2,TEST_00002,Caravaggio
3,TEST_00003,Albrecht Du rer
4,TEST_00004,Vincent van Gogh


In [28]:
submit.to_csv('./csv/efficientnet_b4_v3.csv', index=False)